#  RAG 체인 구성
- Naïve RAG 구현

### **학습 목표:**  RAG 기반의 질의응답 시스템을 구현할 수 있다

### **실습 자료**: 

- data/transformer.pdf

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) 문서 로드`

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 초기화
pdf_loader = PyPDFLoader('./data/transformer.pdf')

# 동기 로딩
pdf_docs = pdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

`(4) 텍스트 분할`

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# Hugging Face의 임베딩 모델 생성
embeddings_huggingface = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# 토크나이저 직접 접근
tokenizer = embeddings_huggingface._client.tokenizer

# 토크나이저를 사용한 예시
text = "테스트 텍스트입니다."
tokens = tokenizer(text)
print(tokens)

# 토크나이저 설정 확인
print(tokenizer.model_max_length)  # 최대 토큰 길이
print(tokenizer.vocab_size)        # 어휘 크기

In [ ]:
# 토큰 수를 계산하는 함수
def count_tokens(text):
    return len(tokenizer(text)['input_ids'])

# 토큰 수 계산
text = "테스트 텍스트입니다."
print(count_tokens(text))

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 생성
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,                      
    chunk_overlap=100,           
    length_function=count_tokens,         # 토큰 수를 기준으로 분할
    separators=["\n\n", "\n", " ", ""],   # 구분자 - 재귀적으로 순차적으로 적용 
)

# 텍스트 분할
chunks = text_splitter.split_documents(pdf_docs)
print(f"생성된 텍스트 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")
print(f"각 청크의 토큰 수: {list(count_tokens(chunk.page_content) for chunk in chunks)}")

In [ ]:
# 청크의 텍스트 확인
print(chunks[2].page_content)

# 벡터 저장소 기반 RAG 검색기 (Retriever)



`(1) 벡터 저장소 초기화`
- chroma 사용
- cosine distance 기준으로 인덱싱 

In [ ]:
from langchain_chroma import Chroma

# Chroma 벡터 저장소 생성하기
chroma_db = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings_huggingface,    # huggingface 임베딩 사용
    collection_name="db_transformer",    # 컬렉션 이름
    persist_directory="./chroma_db",
    collection_metadata = {'hnsw:space': 'cosine'}, # l2, ip, cosine 중에서 선택 
)

# 현재 저장된 컬렉션 데이터 확인
chroma_db.get()

`(2) Top K`

In [ ]:
chroma_k_retriever = chroma_db.as_retriever(
    search_kwargs={"k": 2},
)

query = "대표적인 시퀀스 모델은 어떤 것들이 있나요?"
retrieved_docs = chroma_k_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"-{i}-\n{doc.page_content[:100]}...{doc.page_content[-100:]} [출처: {doc.metadata['source']}]")
    print("-" * 100)

`(3) 임계값 지정`
- Similarity score threshold (기준 스코어 이상인 문서를 대상으로 추출)

In [ ]:
from langchain_community.utils.math import cosine_similarity

chroma_threshold_retriever = chroma_db.as_retriever(
    search_type='similarity_score_threshold',       # cosine 유사도
    search_kwargs={'score_threshold': 0.5, 'k':2},  # 0.5 이상인 문서를 추출
)

query = "대표적인 시퀀스 모델은 어떤 것들이 있나요?"
retrieved_docs = chroma_threshold_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    score = cosine_similarity(
        [embeddings_huggingface.embed_query(query)], 
        [embeddings_huggingface.embed_query(doc.page_content)]
        )[0][0]
    print(f"-{i}-\n{doc.page_content[:100]}...{doc.page_content[-100:]} [유사도: {score}]")
    print("-" * 100)

`(4) MMR(Maximal Marginal Relevance) 검색`

In [ ]:
# MMR - 다양성 고려 (lambda_mult 작을수록 더 다양하게 추출)
chroma_mmr = chroma_db.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k': 3,                 # 검색할 문서의 수
        'fetch_k': 8,           # mmr 알고리즘에 전달할 문서의 수 (fetch_k > k)
        'lambda_mult': 0.5,     # 다양성을 고려하는 정도 (1은 최소 다양성, 0은 최대 다양성을 의미. 기본값은 0.5)
        },
)


query = "대표적인 시퀀스 모델은 어떤 것들이 있나요?"
retrieved_docs = chroma_mmr.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    score = cosine_similarity(
        [embeddings_huggingface.embed_query(query)], 
        [embeddings_huggingface.embed_query(doc.page_content)]
        )[0][0]
    print(f"-{i}-\n{doc.page_content[:100]}...{doc.page_content[-100:]} [유사도: {score}]")
    print("-" * 100)

`(5) metadata 필터링 검색`

In [ ]:
# 메타데이터 확인
chunks[0].metadata

In [ ]:
# 문서 객체의 metadata를 이용한 필터링
chrom_metadata = chroma_db.as_retriever(
    search_kwargs={
        'filter': {'source': './data/transformer.pdf'},
        'k': 5, 
        }
)

query = "대표적인 시퀀스 모델은 어떤 것들이 있나요?"
retrieved_docs = chrom_metadata.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"-{i}-\n{doc.page_content} [출처: {doc.metadata['source']}]")
    print("-" * 100)

`(6) page_content 본문 필터링 검색`

In [ ]:
# page_content를 이용한 필터링
chroma_content = chroma_db.as_retriever(
    search_kwargs={
        'k': 2,
        'where_document': {'$contains': 'recurrent'},
        }
)

query = "대표적인 시퀀스 모델은 어떤 것들이 있나요?"
retrieved_docs = chroma_content.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"-{i}-\n{doc.page_content} [출처: {doc.metadata['source']}]")
    print("-" * 100)

# [실습 프로젝트] Naive RAG 구현 

- 각 단계별 지시사항에 따라 코드를 완성하세요. 
- 제시된 지시사항과 LangChain 문서를 참조하여 시스템을 구성합니다. 

`(1) 벡터 저장소 설정`
- HuggingFace에서 지원하는 BAAI/bge-m3 임베딩 모델을 사용하여 문서를 벡터화
- FAISS DB를 벡터 스토어로 사용 (IndexFlatL2 사용: 유클리드 거리)

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings  

# Hugging Face의 임베딩 모델 생성
embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# 임베딩 차원 확인
embedding = embeddings_model.embed_query("test")
print(f"임베딩 차원: {len(embedding)}")

In [ ]:
# Ollama 임베딩 모델을 사용한 FAISS 벡터 저장소 생성
import faiss 
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

# FAISS 인덱스 초기화 (유클리드 거리 사용)
dim = 1024  # 임베딩 차원
faiss_index = faiss.IndexFlatL2(dim)  

# FAISS 벡터 저장소 생성
faiss_db = FAISS(
    embedding_function=embeddings_model,
    index=faiss_index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

# 저장된 문서의 갯수 확인
print(faiss_db.index.ntotal)

In [ ]:
import uuid

# 문서 id 생성
doc_ids = [str(uuid.uuid4()) for _ in range(len(chunks))]

# 문서를 벡터 저장소에 저장
added_doc_ids = faiss_db.add_documents(chunks, ids=doc_ids)

# 벡터 저장소에 저장된 문서를 확인
print(f"{len(added_doc_ids)}개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.")
print(added_doc_ids)

`(2) 검색기 정의`
- mmr 검색으로 상위 3개 문서 검색하는 Retriever 사용
- 다양성을 높이는 설정을 사용 

In [ ]:
# mmr 검색기 생성
faiss_mmr_retriever = faiss_db.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k': 3,                 # 검색할 문서의 수
        'fetch_k': 8,           # mmr 알고리즘에 전달할 문서의 수 (fetch_k > k)
        'lambda_mult': 0.3,     # 다양성을 높이는 설정 (0에 가까울수록 더 다양함)
    }
)

In [ ]:
# 검색 테스트 
query = "대표적인 시퀀스 모델은 어떤 것들이 있나요?"
retrieved_docs = faiss_mmr_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"-{i}-\n{doc.page_content[:100]}...{doc.page_content[-100:]}")
    print("-" * 100)

`(3) RAG 프롬프트 구성`

- 작성 기준: 
    - LangChain의 ChatPromptTemplate 클래스 사용
    - 변수 처리는 {context}, {question} 형식 사용
    - 답변은 한글로 출력되도록 프롬프트 작성
    
- 아래 템플릿 코드를 기반으로 다음 내용을 참고하여 작성합니다. 

    1. 프롬프트 구성요소:
        - 작업 지침
        - 컨텍스트 영역
        - 질문 영역
        - 답변 형식 가이드

    2. 작업 지침:
        - 컨텍스트 기반 답변 원칙
        - 외부 지식 사용 제한
        - 불확실성 처리 방법
        - 답변 불가능한 경우의 처리 방법

    3. 답변 형식:
        - 핵심 답변 섹션
        - 근거 제시 섹션
        - 추가 설명 섹션 (필요시)

    4. 제약사항 반영:
        - 답변은 사실에 기반해야 함
        - 추측이나 가정을 최소화해야 함
        - 명확한 근거 제시가 필요함
        - 구조화된 형태로 작성되어야 함

In [52]:
# Prompt 템플릿 (예시)
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context.

[Context]
{context}

[Question] 
{question}

[Answer]
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
# Prompt 템플릿 (여기에 작성하세요)
from langchain.prompts import ChatPromptTemplate

template = """다음 컨텍스트를 기반으로 질문에 답하세요. 컨텍스트에서 찾을 수 없는 내용은 추측하지 말고, 모르겠다고 답하세요.

[작업 지침]
- 주어진 컨텍스트의 정보만을 사용하여 답변하세요
- 외부 지식을 사용하지 마세요
- 불확실한 정보는 추측하지 말고 명확히 표시하세요
- 답변할 수 없다면 솔직히 말하세요

[컨텍스트]
{context}

[질문]
{question}

[답변]
위 컨텍스트를 바탕으로 답변드리겠습니다:

**핵심 답변:**

**근거:**

**추가 설명 (해당되는 경우):**

답변은 한국어로 작성하며, 사실에 기반하여 명확하게 제시하겠습니다."""

prompt = ChatPromptTemplate.from_template(template)

# 템플릿 출력
prompt.pretty_print()

`(4) RAG 체인 구성`
- LangChain의 LCEL 문법을 사용
- 검색 결과를 프롬프트의 'context'로 전달하고,
- 사용자가 입력한 질문을 그래도 프롬프트의 'question'에 전달
- LLM 설정:
    - ChatOpenAI 사용 ('gpt-4.1-mini' 모델)
    - temperature: 답변의 일관성을 가져가는 설정값을 사용 
    - 기타 필요한 설정 
- 출력 파서: 문자열 부분만 출력되도록 구성 

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# LLM 설정
llm = ChatOpenAI(
    model="gpt-4.1-mini", 
    temperature=0  # 일관성을 위한 설정
)

# 문서 포맷팅
def format_docs(docs):
    return "\n\n".join([f"{doc.page_content}" for doc in docs])

# RAG 체인 생성
rag_chain = (
    {
        "context": faiss_mmr_retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt 
    | llm 
    | StrOutputParser()
)

# 체인 실행
query = "대표적인 시퀀스 모델은 어떤 것들이 있나요?"
output = rag_chain.invoke(query)

print(f"쿼리: {query}")
print("답변:")
print(output)

`(5) Gradio 스트리밍 구현`
- ChatInterface 사용
- `chain.stream()`으로 응답을 청크 단위로 스트리밍

In [ ]:
import gradio as gr
from typing import Iterator

# 스트리밍 응답 생성 함수
def get_streaming_response(message: str, history) -> Iterator[str]:
    
    # RAG Chain 실행 및 스트리밍 응답 생성
    response = ""
    for chunk in rag_chain.stream(message):
        if isinstance(chunk, str):
            response += chunk
            yield response

# Gradio 인터페이스 설정
demo = gr.ChatInterface(
    fn=get_streaming_response,
    title="RAG 기반 질의응답 시스템",
    description="Transformer PDF 문서를 기반으로 질문에 답변합니다.",
    examples=[
        "대표적인 시퀀스 모델은 어떤 것들이 있나요?",
        "Transformer의 주요 특징은 무엇인가요?", 
        "어텐션 메커니즘이란 무엇인가요?",
        "인코더-디코더 구조에 대해 설명해주세요."
    ],
    cache_examples=False
)

# 실행
demo.launch()

In [ ]:
# demo 실행 종료
demo.close()